In [14]:

# libraries

import pandas as pd
from ipyvizzu import Chart, Data, Config, Style, ChartProperty
from ipyvizzustory import Story, Slide, Step


## Aggregate demographics

In [61]:

# read old pop data
pop_old = pd.read_csv('./data/uk_adult_pop.csv')
pop_old['sexual_orientation'] = "Unknown"

# read 2021 data
pop_2021 = pd.read_excel('data\msoa_pop_by_orientation.xlsx')[['Sexual orientation (6 categories)', 'Observation']]

# cleaning
pop_2021 = pop_2021.rename(columns = {'Sexual orientation (6 categories)' : 'sexual_orientation', 'Observation' : 'population'})
pop_2021 = pop_2021.groupby(['sexual_orientation']).sum().reset_index()
pop_2021 = pop_2021[pop_2021['sexual_orientation'] != 'Does not apply']
pop_2021 = pop_2021.replace('All other sexual orientations', 'Other')
pop_2021['year'] = 2021

# append
pop = pop_old._append(pop_2021).sort_values('year')
# pop['year'] = pop['year'].astype(int)
pop['year'] = pop['year'].apply(lambda x: '01-01-' + str(x))
pop['year'] = pd.to_datetime(pop['year'])

# change units
pop['population'] = pop['population'].apply(lambda x: x/1000000).round(2)
pop = pop.rename(columns = {'population' : 'Population (M)',
                            'sexual_orientation' : 'Sexual Orientation',
                            'year' : 'Year'})

pop

,Year,Population (M),Sexual Orientation
5,1961-01-01,34.91,Unknown
4,1971-01-01,36.82,Unknown
3,1981-01-01,38.72,Unknown
2,1991-01-01,40.50,Unknown
1,2001-01-01,41.86,Unknown
0,2011-01-01,45.58,Unknown
0,2021-01-01,0.17,Other
1,2021-01-01,0.62,Bisexual
3,2021-01-01,0.75,Gay or Lesbian
4,2021-01-01,3.63,Not answered


In [ ]:
data = Data()
data.add_df(pop)

# chart

story = Story(data)

slide1 = Slide(
    Step(
        Data.filter("record.Year != '2021'"),
        Config.stackedColumn({"x": "Year",
                "y": "Population (M)",
                "title": "Adult (16+) population in England and Wales",
                "stackedBy": "Sexual Orientation"
                })
    )
)
story.add_slide(slide1)

slide2 = Slide()

slide2.add_step(
    Step(
        Data.filter("record.Year <= '2021'")
))

slide2.add_step(
    Step(
        Data.filter("record.Year == '2021'"),
        Config({
            'y' : 'Sexual Orientation',
            'x' : 'Population (M)',
            'coordSystem' : 'polar',
            'title': '2021 breakdown of adult sexual orientation in England and Wales'
                })
    )
)

story.add_slide(slide2)



story.play()

In [ ]:

data = Data()
data.add_df(pop)


chart = Chart(width=600, height= 400)
chart.animate(data)


chart.animate(Config({"title": "For the first time in over 200 years of censuses..."}))

# chart.feature("tooltip", True)

# chart.animate(
#     Config({"channels": {"label": {"attach": ["Population (M)"]}}})
# )

# set axis limits

# Set axis
chart.animate(
    Config(
        {
            "channels": {
                "y": {"set": ["Population (M)"]},
                "x": {"set": ["Year"]},
            }
        }
    )
)

# # def axis_limit(chart, axis, min, max):
# #     chart.animate(
# #         Config(
# #             {
# #                 "channels": {
# #                     axis: {
# #                         "range": {
# #                             "min": min,
# #                             "max": max,
# #                         }
# #                     }
# #                 }
# #             }
# #         ),
# #     duration = 2
# #     )

# # axis_limit(chart, "x", 1961, 1972)
# # axis_limit(chart, "x", 1961, 1982)
# # axis_limit(chart, "x", 1961, 1992)
# # axis_limit(chart, "x", 1961, 2002)
# # axis_limit(chart, "x", 1961, 2012)
# # axis_limit(chart, "x", 1961, 2022)

# # # Stacking
# # chart.animate(
# #     Config(
# #         {
# #             "channels": {
# #                 "y": {"attach": ["Sexual Orientation"]},
# #                 "color": {"attach": ["Sexual Orientation"]},
# #             }
# #         }
# #     )
# # )


# # # Title
# # chart.animate(Config({"title": """ipyvizzu offers different options to align your chart elements and to set the range of values shown on the axes. Alignment can be used to create charts like a stream chart where the elements are vertically centered. A good example for using range is when you fix the y-axis so that it would not adapt to the data being shown.

# # Centered alignment. The effect of this parameter depends on the orientation of the chart. For example, on a column chart, elements will be vertically centered, whereas on a bar chart, horizontally."""}))

# # chart.animate(
# #     Config(
# #         {
# #             "channels": {"color": {"attach": ["Sexual Orientation"]}},
# #             "legend": "color",
# #         }
# #     )
# # )


# # chart.animate(
# #     Config(
# #         {
# #             "channels": {
# #                 "y": {"set": ["Population (M)","Year"]},
# #                 "x": {"set": None},
# #             }
# #         }
# #     )
# # )


# # # Add/remove from axis
# # chart.animate(
# #     Config(
# #         {
# #             "channels": {
# #                 "y": {"detach": ["Population (M)", "Year"]},
# #                 "x": {"attach": ["Population (M)"]},
# #                 "y": {"attach": ["Year"]},
# #             }
# #         }
# #     )
# # )




# # # Change to circles
# # chart.animate(
# #     Config(
# #         {
# #             "channels": {"size": {"set": ["Population (M)"]}},
# #             "geometry": "circle",
# #         }
# #     )
# # )


# # # delay and duration
# # chart.animate(
# #     Config(
# #         {
# #             "title": "This is the breakdown",
# #             "channels": {
# #                 "y": {"detach": ["Year"]},
# #                 "x": {"attach": ["Year"]},
# #             }
# #         }
# #     ),
# #     duration=4,
# #     delay=2
# # )

# # # remove labels
# # chart.animate(
# #     Config({"channels": {"y": {"labels": False},
# #                          "x": {"labels": False}}})
# # )

# # # filter

# # filter1 = Data.filter(
# #     """
# #     record['Sexual Orientation'] == 'Gay or Lesbian' || 
# #     record['Sexual Orientation'] == 'Bisexual'
# #     """
# # )

# # chart.animate(filter1)

# # chart.animate(Data.filter(None))






In [82]:
# styling
data = Data()
data.add_df(pop)


chart = Chart(width=600, height= 400)
chart.animate(data)


chart.animate(Config({"title": "For the first time in over 200 years of censuses..."}))

chart.animate(
    Config(
        {
            "channels": {
                "y": {"set": ["Population (M)"]},
                "x": {"set": ["Year"]},
            }
        }
    )
)

style = Style(
    {
        "backgroundColor": "#12273F",
        "fontFamily": "Century Gothic, sans-serif",
        "fontSize": "10px",

        "legend": {
            "label": {
                "color": "#999999FF",
                "fontSize": "1em",
                "textAlign": "left"
            },
            "title": {
                "color": "#999999FF",
                "fontSize": "1.166667em",
                "textAlign": "left"
            }
        },

        "plot": {
            "marker": {
                "colorPalette": "#3CD7D9 #FF7300 #9E71FE #D4AF37 #A5D700 #FF50C8 #5297FF"
            },
            "yAxis": {
                "color": "#FFFFFF",
                "label": {
                    "color": "#FFFFFF",
                    "fontSize": "1em"
                },
                "title": {
                    "color": "#FFFFFF",
                    "fontSize": "1.166667em"
                },
            },
            "xAxis": {
                "label": {
                    "color": "#FFFFFF",
                    "fontSize": "1em"
                },
                "title": {
                    "color": "#FFFFFF",
                    "fontSize": "1.166667em"
                },
            },
        },

        "title": {
            "color": "#FFFFFF",
            "fontSize": "2.166667em",
            "textAlign": "center"
        },

        "tooltip": {
            "arrowSize": "8",
            "color": "#191919FF",
            "backgroundColor": "#FFFFFFFF",
            "borderColor": "#D8D8D8FF",
            "fontSize": "12px"
        },

        "logo": { 
            'filter' : 'color(#D8D8D8FF)',
        },
    }
)


chart.animate(style)


chart.log


<bound method Chart.log of <ipyvizzu.chart.Chart object at 0x000001C0D827CC70>>